# WheatDet Augmentation Turkce Aciklamali

Augmentation daha çok projelerde ezberleme (overfitting) problemini çözmek ve başarımı artırmak için kullanılır. Yapay sinir ağlarındaki temel optimizasyon algoritması olan gradient descent ( momentumlar dahil) lokal min - max'a takılabilir. Bunları aşmanın güzel bir yolu datayı artırmak ve artırırken de farklı yapılar kullanmaktır. Bu yapılar, fotoğrafı döndürmek, fotoğrafa yakınlaşmak veya uzaklaşmak, üstünden parçalar çıkarmak, üstüne parçalar ilave etmek, gürültü eklemek, değerleriyle oynamak, renk kanallarından bazılarını çıkarmak v.b olabilir. En ama en önemli nokta: projenin ilgili olduğu konuda, gerçek durumda karşılaşılabilecek örneklere göre augmentation yapmaktır. 

Örneğin araba tespiti yapıyorsunuz diyelim. Arabayı vertical flip ile dikey aynalamanız güzel bir augmentationdır çünkü gerçek durumda arabalar her iki yönde de gidebilir. Lakin horizantal flip yapmak yani arabayı baş aşağı yapmak iyi bir durum olmayacaktır çünkü gerçek hayatta araçlar lastikleri üstünde giderler :)

Albumination kütüphanesi ile data augmentation ( Data artırımı )konusunu bu çalışmada inceleyeceğiz. Yararlandığımız makale aşağıdadır, okumanızı tavsiye ederim:


Albumentations: Fast and Flexible Image Augmentations

Buslaev, A., Iglovikov, V. I., Khvedchenya, E., Parinov, A., Druzhinin, M., & Kalinin, A. A. (2020). Albumentations: Fast and Flexible Image Augmentations. Information, 11(2), 125. doi:10.3390/info11020125 

> import albumentations


In [ ]:
import os
import ast
from collections import namedtuple

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm
from PIL import Image

import joblib
from joblib import Parallel, delayed

import cv2
import albumentations
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations.bbox_utils import denormalize_bbox, normalize_bbox

from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib.image import imsave

Aşağıdaki kütüphaneler, object detection için daha çok.

In [ ]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")

import torch
import torchvision
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2, ToTensor

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from tqdm.notebook import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from matplotlib import pyplot as plt

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

Fotoğrafları görselleştiriyoruz. 6 adet random fotoğraf çağırıyoruz. 

In [ ]:
train_folder_path = "/kaggle/input/global-wheat-detection/train/"
test_folder_path = "/kaggle/input/global-wheat-detection/test/"

dirlist  = os.listdir(train_folder_path)
dirtlist =  os.listdir(test_folder_path)
a=len(dirlist)
b=len(dirtlist)
print("# of train sample: " , a)
print("# of test sample: " , b)

a=random.randint(0,a-1)
b=random.randint(0,b-1)
c=random.randint(0,a)
d=random.randint(0,a)

im = cv2.imread(os.path.join(train_folder_path, dirlist[a]))
im1 = cv2.imread(os.path.join(train_folder_path, dirlist[a+b]))
im3 = cv2.imread(os.path.join(train_folder_path, dirlist[a-b]))
im2 = cv2.imread(os.path.join(test_folder_path, dirtlist[b]))
im4 = cv2.imread(os.path.join(train_folder_path, dirlist[c+b]))
im5 = cv2.imread(os.path.join(train_folder_path, dirlist[d+b]))
im6 = cv2.imread(os.path.join(train_folder_path, dirlist[c-+b]))


print("shape of the train photo: ", im.shape)
print("shape of the test photo: ", im2.shape)





fig = plt.figure()


f, ax = plt.subplots(2,3,figsize=(20, 20)) 

ax[0][0].imshow(im)
ax[1][0].imshow(im2)
ax[1][1].imshow(im1)
ax[0][1].imshow(im3)
ax[1][2].imshow(im5)
ax[0][2].imshow(im4)


plt.show()


2 tür bounding box vardır. Pascal-VOC ve COCO. Temel olarak aynı yapılardır sadece bounding box kaydedilirken farklı metrikleri saklamaktadırlar. İnternette birbirine çevrim kodları mevcuttur.

pascal_voc yapısı :  [x_min, y_min, x_max, y_max]

COCO yapısı :  [x_min, y_min, width, height]

Data frame yapısındaki bbox listesini 4 ayrı sütuna bölüyoruz. Gördüğünüz gibi xmin, ymin, width, height şeklindeki yapı bize COCO bounding box yapısına işaret ediyor.



In [ ]:
# Constants
BASE_DIR = '/kaggle/input/global-wheat-detection'
WORK_DIR = '/kaggle/working'


train_df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'))
print("Initial DF shape: ", train_df.shape)
train_df.head(6)

# Let's expand the bounding box coordinates and calculate the area of all the bboxes
train_df[['x_min','y_min', 'width', 'height']] = pd.DataFrame([ast.literal_eval(x) for x in train_df.bbox.tolist()], index= train_df.index)
train_df = train_df[['image_id', 'bbox', 'source', 'x_min', 'y_min', 'width', 'height']]
train_df['area'] = train_df['width'] * train_df['height']
train_df['x_max'] = train_df['x_min'] + train_df['width']
train_df['y_max'] = train_df['y_min'] + train_df['height']
train_df = train_df.drop(['bbox', 'source'], axis=1)
train_df = train_df[['image_id', 'x_min', 'y_min', 'x_max', 'y_max', 'width', 'height', 'area']]

# There are some buggy annonations in training images having huge bounding boxes. Let's remove those bboxes
train_df = train_df[train_df['area'] < 100000]

print("Final DF shape:",train_df.shape)
train_df.head(6)

Bir adet fotoğraf çağırıyoruz. BGR kanalları RGB'ye çeviriyoruz. Daha sonrasında numpy float array'e dönüştürüyoruz.
Bir başka adım ise ( her zaman gerekli değil ama daha küçük sayılarla uğraşmak için iyi bir yöntem) değerleri 255'e bölerek normalize ediyoruz. Bildiğiniz gibi RGB bir imgenin her bir renk kanalındaki pixeller 0 ile 255 arasında değer alabilir.

data frame --> np array'e çevriliyor:  "  astype(np.int32).values "

In [ ]:
# Read the image on which data augmentaion is to be performed

image = cv2.imread(os.path.join(train_folder_path, dirlist[a]),cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
image /= 255.0
plt.figure(figsize = (10, 10))
plt.imshow(image)
plt.show()
image_id = dirlist[a][:-4]
print(image_id)

Fotoğrafın üstüne bounding box'ı koyuyoruz. Çok basit bir işlem. İster pascal-voc, ister coco olsun, koordinatları alıp, fotoğrafın üstüne dikdörtgen çiziyoruz. 

In [ ]:
pascal_voc_boxes = train_df[train_df['image_id'] == image_id][['x_min', 'y_min', 'x_max', 'y_max']].astype(np.int32).values
pascal_voc_boxes.shape
coco_boxes = train_df[train_df['image_id'] == image_id][['x_min', 'y_min', 'width', 'height']].astype(np.int32).values
coco_boxes.shape
assert(len(pascal_voc_boxes) == len(coco_boxes))
labels = np.ones((len(pascal_voc_boxes), ))

get_bbox fonksiyonu object detection için augmentation yaparken en önemli yapılardan birisidir. Ana fotoğrafa her nasıl augmentation uyguluyorsanız, bounding box'lar da benzer şekilde değişmelidir.

In [ ]:
def get_bbox(bboxes, col, color='white', bbox_format='pascal_voc'):
    
    for i in range(len(bboxes)):
        # Create a Rectangle patch
        if bbox_format == 'pascal_voc':
            rect = patches.Rectangle(
                (bboxes[i][0], bboxes[i][1]),
                bboxes[i][2] - bboxes[i][0], 
                bboxes[i][3] - bboxes[i][1], 
                linewidth=2, 
                edgecolor=color, 
                facecolor='none')
        else:
            rect = patches.Rectangle(
                (bboxes[i][0], bboxes[i][1]),
                bboxes[i][2], 
                bboxes[i][3], 
                linewidth=2, 
                edgecolor=color, 
                facecolor='none')

        # Add the patch to the Axes
        col.add_patch(rect)

# Resize, Random Crpo ve Dikey döndürme

In [ ]:
#Random Crop & VerticalFlip

aug = albumentations.Compose([
        albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.VerticalFlip(1),    # Verticlly flip the image
        albumentations.RandomCrop(height=350, width=350, p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

aug_result = aug(image=image, bboxes=pascal_voc_boxes, labels=labels)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 16))
get_bbox(pascal_voc_boxes, ax[0], color='red')
ax[0].title.set_text('Original Image')
ax[0].imshow(image)

get_bbox(aug_result['bboxes'], ax[1], color='red')
ax[1].title.set_text('Augmented Image')
ax[1].imshow(aug_result['image'])
plt.show()

# Gaussian Gürültü ekleme ve Cutout

In [ ]:
#GaussNoise & Cutout

aug = albumentations.Compose([
        albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
#         albumentations.VerticalFlip(1),    # Verticlly flip the image
        albumentations.Cutout(num_holes=8, max_h_size=26, max_w_size=56, fill_value=0, p=1),
        albumentations.GaussNoise(var_limit=(0.002, 0.002), p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


aug_result = aug(image=image, bboxes=pascal_voc_boxes, labels=labels)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 16))

get_bbox(pascal_voc_boxes, ax[0], color='red')
ax[0].title.set_text('Original Image')
ax[0].imshow(image)

get_bbox(aug_result['bboxes'], ax[1], color='red')
ax[1].title.set_text('Augmented Image')
ax[1].imshow(aug_result['image'])
plt.show()

# Kar Yağdırma

In [ ]:
#Random Snow
aug = albumentations.Compose([
        albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
#         albumentations.VerticalFlip(1),    # Verticlly flip the image
        albumentations.RandomSnow(p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
aug_result = aug(image=image, bboxes=pascal_voc_boxes, labels=labels)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 16))
get_bbox(pascal_voc_boxes, ax[0], color='red')
ax[0].title.set_text('Original Image')
ax[0].imshow(image)

get_bbox(aug_result['bboxes'], ax[1], color='red')
ax[1].title.set_text('Augmented Image')
ax[1].imshow(aug_result['image'])
plt.show()

# Yağmur yağdırma

In [ ]:
#RandomRain
aug = albumentations.Compose([
        albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
#         albumentations.VerticalFlip(1),    # Verticlly flip the image
        albumentations.RandomRain(p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
aug_result = aug(image=image, bboxes=pascal_voc_boxes, labels=labels)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 16))
get_bbox(pascal_voc_boxes, ax[0], color='red')
ax[0].title.set_text('Original Image')
ax[0].imshow(image)

get_bbox(aug_result['bboxes'], ax[1], color='red')
ax[1].title.set_text('Augmented Image')
ax[1].imshow(aug_result['image'])
plt.show()

# Güneş Yansıması Ekleme

In [ ]:
#Random Sun Flake
aug = albumentations.Compose([
        albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
#         albumentations.VerticalFlip(1),    # Verticlly flip the image
        albumentations.RandomSunFlare(flare_roi=(0, 0, 1, 0.4), src_radius=100, src_color=(255, 255, 255), p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
aug_result = aug(image=image, bboxes=pascal_voc_boxes, labels=labels)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 16))
get_bbox(pascal_voc_boxes, ax[0], color='red')
ax[0].title.set_text('Original Image')
ax[0].imshow(image)

get_bbox(aug_result['bboxes'], ax[1], color='red')
ax[1].title.set_text('Augmented Image')
ax[1].imshow(aug_result['image'])
plt.show()